In [1]:
from strokes import StrokePatientsMIDataset
import scipy
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
from torcheeg.transforms import  Select,BandSignal,Compose,Lambda,ToTensor,BandDifferentialEntropy,Resize
from to import ToGrid
from typing import Callable, Dict, Union, List
import numpy as np
import soxr
from downsample import SetSamplingRate
from baseline import BaselineCorrection
from base_transform import EEGTransform
    

dataset = StrokePatientsMIDataset(root_path='../kinlaw12/subdataset',
                        chunk_size=500,  # 1 second
                        overlap = 250,
                        offline_transform=Compose(
                                [BaselineCorrection(),SetSamplingRate(500,484),
                                BandSignal(sampling_rate=484,band_dict={'frequency_range':[8,40]})]),
                        online_transform=Compose(
                                [ToTensor()]),
                
                        label_transform=Select('label'),
                        num_worker=8
)
print(dataset[0][0].shape) #EEG shape(1,30,500)
print(dataset[0][1])  # label (int)
print(len(dataset))

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '../kinlaw12/subdataset/participants.tsv'

In [1]:
from torcheeg.datasets import BCICIV2aDataset
from torcheeg import transforms
from torcheeg.datasets.constants.motor_imagery.bciciv_2a import  BCICIV2A_LOCATION_DICT
from downsample import SetSamplingRate
from baseline import BaselineCorrection

dataset = BCICIV2aDataset(root_path='.././BCIdataset',
                          io_path='.torcheeg/datasets_1733052246128_H21dG',
                          offline_transform=transforms.Compose(
                              [BaselineCorrection(),SetSamplingRate(1750,128)]),
                          online_transform=transforms.Compose([
                              transforms.ToGrid(BCICIV2A_LOCATION_DICT),
                              transforms.ToTensor()
                          ]),
                          label_transform=transforms.Compose([
                              transforms.Select('label'),
                              transforms.Lambda(lambda x: x - 1)
                          ]))
print(dataset[0][0].shape)
# EEG signal (torch.Tensor[1, 22, 1750])
# label (int)

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2024-12-01 21:49:46] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from .torcheeg/datasets_1733052246128_H21dG.


torch.Size([128, 6, 7])


In [2]:
print(dataset[0][0].shape)

torch.Size([1, 30, 484])


In [5]:
import sys
import os

# 将上上级目录添加到 sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))

# 现在可以导入 dwc_codes
from dwc_codes.SEALtrainer import SEALClassifierTrainer

In [2]:
HYPERPARAMETERS = {
    "seed": 42,
    "batch_size": 32,
    "lr": 1e-4,
    "weight_decay": 0,
    "num_epochs": 100,
}

In [3]:
from eca import ECA
from model import SwinTransformer
import torch
import torch.nn as nn

class ECASwinTransformerModel(nn.Module):
    def __init__(self, num_channels):
        super(ECASwinTransformerModel, self).__init__()
        self.eca = ECA(input_size=484,num_attention_heads=2,hidden_size=484,hidden_dropout_prob=0.5)
        self.swin_transformer = SwinTransformer(
                                                in_chans=30,
                                                num_classes=2,
                                                embed_dim=192,
                                                depths=(2, 2, 18, 2),
                                                num_heads=(6, 12, 24, 48),
                                                # patch_size=3,
                                                # window_size=4,
                                                # drop_rate=0.2,
                                                # attn_drop_rate=0.5,
                                                # mlp_ratio=2,
                                                # qkv_bias=True,
                                                # drop_path_rate=0.2,
                                                )

    def forward(self, x):
        # x shape: (batch_size, channels, time)
        x = x.view(x.size(0),30,484)
        x = self.eca(x)
        # Reshape for Swin Transformer
        x = x.view(x.size(0),30,22,22)
        x = self.swin_transformer(x)
        return x

In [3]:
# x = dataset[0][0].unsqueeze(1)
# print(dataset[0][1])
model = ECASwinTransformerModel(num_channels=30)
for name, param in model.named_parameters():
    print(f"Parameter: {name}, requires_grad: {param.requires_grad}")

/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Parameter: eca.query.weight, requires_grad: True
Parameter: eca.query.bias, requires_grad: True
Parameter: eca.key.weight, requires_grad: True
Parameter: eca.key.bias, requires_grad: True
Parameter: eca.value.weight, requires_grad: True
Parameter: eca.value.bias, requires_grad: True
Parameter: eca.dense.weight, requires_grad: True
Parameter: eca.dense.bias, requires_grad: True
Parameter: eca.LayerNorm.weight, requires_grad: True
Parameter: eca.LayerNorm.bias, requires_grad: True
Parameter: swin_transformer.patch_embed.proj.weight, requires_grad: True
Parameter: swin_transformer.patch_embed.proj.bias, requires_grad: True
Parameter: swin_transformer.patch_embed.norm.weight, requires_grad: True
Parameter: swin_transformer.patch_embed.norm.bias, requires_grad: True
Parameter: swin_transformer.layers.0.blocks.0.norm1.weight, requires_grad: True
Parameter: swin_transformer.layers.0.blocks.0.norm1.bias, requires_grad: True
Parameter: swin_transformer.layers.0.blocks.0.attn.relative_position_b

In [4]:
import os
import shutil

def delete_folder_if_exists(target_folder_name):
    # 获取父文件夹中的所有内容
    parent_folder = os.getcwd()
    for folder_name in os.listdir(parent_folder):
        folder_path = os.path.join(parent_folder, folder_name)

        # 检查是否是文件夹并且名称是否匹配
        if os.path.isdir(folder_path) and folder_name == target_folder_name:
            try:
                # 删除目标文件夹
                shutil.rmtree(folder_path)
                print(f"已删除文件夹: {folder_path}")
            except Exception as e:
                print(f"删除文件夹 {folder_path} 时出错: {e}")




In [6]:
from torcheeg.model_selection import KFoldPerSubjectGroupbyTrial
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader
from model import SwinTransformer
from torcheeg.trainers import ClassifierTrainer

k_fold = KFoldPerSubjectGroupbyTrial(
    n_splits=4,
    shuffle=True,
    # split_path='.torcheeg/model_selection_1733055200903_KghEv',
    random_state=42)

training_metrics = []
test_metrics = []

for i, (training_dataset, test_dataset) in enumerate(k_fold.split(dataset)):
    delete_folder_if_exists(target_folder_name='lightning_logs')
    model = ECASwinTransformerModel(num_channels=30)
    trainer = ClassifierTrainer(model=model,
                                num_classes=2,
                                lr=HYPERPARAMETERS['lr'],
                                weight_decay=HYPERPARAMETERS['weight_decay'],
                                metrics=["accuracy"],
                                accelerator="gpu")
    # trainer = SEALClassifierTrainer(model=model,
    #                                 num_classes=2,
    #                                 lr=HYPERPARAMETERS['lr'],
    #                                 weight_decay=HYPERPARAMETERS['weight_decay'],
    #                                 metrics=["accuracy"],
    #                                 accelerator="gpu")
    # training_loader = trainer.prepare_train_dataloader(training_dataset, batch_size=HYPERPARAMETERS['batch_size']) 
     
    training_loader = DataLoader(training_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=HYPERPARAMETERS['batch_size'],
                             shuffle=False)
    # 提前停止回调
    early_stopping_callback = EarlyStopping(
        monitor='train_loss',
        patience=20,  # 提前停止的耐心参数
        mode='min',
        verbose=True
    )
    trainer.fit(training_loader,
                test_loader,
                max_epochs=HYPERPARAMETERS['num_epochs'],
                callbacks=[early_stopping_callback],
                enable_progress_bar=True,
                enable_model_summary=False,
                limit_val_batches=0.0)
    # training_result = trainer.test(training_loader,
    #                                enable_progress_bar=True,
    #                                enable_model_summary=True)[0]
    test_result = trainer.test(test_loader,
                               enable_progress_bar=True,
                               enable_model_summary=True)[0]
    # training_metrics.append(training_result["test_accuracy"])
    test_metrics.append(test_result["test_accuracy"])
     
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

[2024-12-02 19:44:32] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-12-02 19:44:32] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1733139872550_ATyOL for the next run, if you want to use the same setting for the experiment.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /root/autodl-tmp/.autodl/kinlaw/mi_swin/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/root/autodl-tmp/conda/env

Epoch 0: 100%|██████████| 7/7 [00:01<00:00,  5.25it/s, loss=2.36, v_num=0, train_loss=0.534, train_accuracy=0.750] 

Metric train_loss improved. New best score: 0.534


Epoch 0: 100%|██████████| 7/7 [00:01<00:00,  5.24it/s, loss=2.36, v_num=0, train_loss=0.534, train_accuracy=0.750]

[2024-12-02 19:44:43] INFO (torcheeg/MainThread) 
[Train] train_loss: 2.357 train_accuracy: 0.550 



...

Epoch 41: 100%|██████████| 8/8 [00:04<00:00,  1.81it/s, loss=0.085, v_num=0, train_loss=0.146, train_accuracy=0.875]


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/autodl-tmp/conda/envs/law/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 47.15it/s]

[2024-12-02 21:30:37] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.533 test_accuracy: 0.875 



Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 41.62it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy                0.875
        test_loss           1.5329638719558716
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'test_metric_avg': 0.7046874985098839, 'test_metric_std': 0.1042590672908914}


In [8]:
for i, score in enumerate(test_metrics):
    print(i,score)

0 0.5375000238418579
1 0.6625000238418579
2 0.637499988079071
3 0.6499999761581421
4 0.6000000238418579
5 0.7124999761581421
6 0.574999988079071
7 0.675000011920929
8 0.6000000238418579
9 0.6875
10 0.7749999761581421
11 0.7749999761581421
12 0.5
13 0.6875
14 0.699999988079071
15 0.7250000238418579
16 0.7250000238418579
17 0.699999988079071
18 0.6875
19 0.7749999761581421
20 0.862500011920929
21 0.8999999761581421
22 0.887499988079071
23 0.875


In [9]:

# 每4个计算平均值
averages = [sum(test_metrics[i:i+4]) / len(test_metrics[i:i+4]) for i in range(0, len(test_metrics), 4)]

print("每4个数的平均值:", averages)

每4个数的平均值: [0.6218750029802322, 0.640625, 0.7093749940395355, 0.6531250029802322, 0.7218749970197678, 0.8812499940395355]


In [7]:
import numpy as np
print({
    # "training_metric_avg": np.mean(training_metrics),
    # "training_metric_std": np.std(training_metrics),
    "len": len(test_metrics),
    "test_metric_avg": np.mean(test_metrics),
    "test_metric_std": np.std(test_metrics)
})

{'len': 24, 'test_metric_avg': 0.7046874985098839, 'test_metric_std': 0.1042590672908914}


In [ ]:
from torcheeg.model_selection import KFoldCrossSubject, LeaveOneSubjectOut, KFoldPerSubject, KFoldPerSubjectGroupbyTrial
from torch.utils.data import DataLoader
cv = KFoldPerSubjectGroupbyTrial(n_splits=5, shuffle=True)
for i, (train_dataset, val_dataset) in enumerate(cv.split(dataset)):
      # The total number of experiments is the number subjects multiplied by K
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

[2024-11-17 15:35:25] INFO (torcheeg/MainThread) 📊 | Create the split of train and test set.
[2024-11-17 15:35:25] INFO (torcheeg/MainThread) 😊 | Please set split_path to .torcheeg/model_selection_1731828925571_IMilb for the next run, if you want to use the same setting for the experiment.


In [ ]:
import os
import argparse
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from utils_swin import train_one_epoch, evaluate
from model import SwinTransformer
from torcheeg.model_selection import KFoldCrossSubject, LeaveOneSubjectOut, KFoldPerSubject, KFoldPerSubjectGroupbyTrial

def main(args):
    # 设置设备
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    if not os.path.exists("./weights"):
        os.makedirs("./weights")

    # DataLoader
    batch_size = args.batch_size
    # train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    # val_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4)
   
    # Model
    # model = create_model(img_size=9, in_chans=5, num_classes=3, patch_size=3, window_size=3).to(device)
    model = SwinTransformer(
                        img_size=7,           # 输入的空间尺寸
                        patch_size=2,         # 每个补丁为 3x3
                        in_chans=500,           # 通道数为4
                        num_classes=2,        # 3分类
                        window_size=3         # 窗口大小3x3
                    ).to(device)
    # Optionally freeze layers
    if args.freeze_layers:
        for name, param in model.named_parameters():
            if "head" not in name:
                param.requires_grad_(False)

    # Optimizer
    pg = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.AdamW(pg, lr=args.lr, weight_decay=5e-2)
    max_acc = 0.5
    cv = KFoldPerSubject(n_splits=10, shuffle=True)
    for i, (train_dataset, val_dataset) in enumerate(cv.split(dataset)):
        # The total number of experiments is the number subjects multiplied by K
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
        for epoch in range(args.epochs):
            # Train
            train_loss, train_acc = train_one_epoch(model=model, optimizer=optimizer, data_loader=train_loader, device=device, epoch=epoch)
        
            # Validate
        val_loss, val_acc = evaluate(model=model, data_loader=val_loader, device=device, epoch=epoch)
        if val_acc > max_acc:
            max_acc = val_acc
            # Save model
            torch.save(model.state_dict(), f"./weights/model-{i}-{epoch}_0876.pth")    

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--num_classes', type=int, default=2)
    parser.add_argument('--epochs', type=int, default=50)
    parser.add_argument('--batch-size', type=int, default=16)
    parser.add_argument('--lr', type=float, default=0.0001)
    parser.add_argument('--freeze-layers', type=bool, default=False)
    parser.add_argument('--device', default='cuda:0', help='device id (i.e. 0 or 0,1, cpu)')

    opt = parser.parse_args()

    main(opt)

In [11]:
from strokes import StrokePatientsMIProcessedDataset
from strokesdict import STROKEPATIENTSMI_LOCATION_DICT
from baseline import BaselineCorrection
from torcheeg.transforms import BandDifferentialEntropy, ToGrid, ToTensor, Select, Compose
from torcheeg.model_selection import KFoldPerSubject
from torch.utils.data import DataLoader
dataset_predict = StrokePatientsMIProcessedDataset(root_path='./strokedataset_TEST',
                                                    # io_path='.torcheeg/datasets_1732260282757_rcS0M',
                                                    chunk_size=500,  # 1 second
                                                    overlap = 0,
                                                    offline_transform=Compose(
                                                        [
                                                        BaselineCorrection(),ToGrid(STROKEPATIENTSMI_LOCATION_DICT)]),
                                                    online_transform=Compose([ToTensor()]),
                                                    label_transform=Select('label'),
                                                    num_worker=8
)

# dataset_predict = dataset
# 存储所有数据段
predict_datalist = []
predict_labellist = []
predict_subjectlist = []
cv = KFoldPerSubject(n_splits=5, shuffle=True)
# data_predict = DataLoader(dataset_predict, batch_size=1, shuffle=False)
for i in range(11,13):
    data = cv.split(dataset_predict, subject=f"{i}")
    for t,v in data:
        vl = DataLoader(v, batch_size=1, shuffle=False)
        for d, l in vl:
            predict_datalist.append(d)
            predict_labellist.append(l)
            predict_subjectlist.append(i)

# 遍历数据并累积
# for i, (input_data, labels) in enumerate(data_predict):
#     # print(input_data[1,0,:,:], labels)
#     if i%50 == 0: 
#         predict_datalist.append(input_data)
#         predict_labellist.append(labels)
print(len(predict_datalist))

[2024-11-22 15:29:24] INFO (torcheeg/MainThread) 🔍 | Processing EEG data. Processed EEG data has been cached to .torcheeg/datasets_1732260564718_uh536.
[2024-11-22 15:29:24] INFO (torcheeg/MainThread) ⏳ | Monitoring the detailed processing of a record for debugging. The processing of other records will only be reported in percentage to keep it clean.
[PROCESS]: 100%|██████████| 2/2 [00:00<00:00, 2364.32it/s]

[RECORD ./strokedataset_TEST/edffile/sub-12/eeg/sub-12_task-motor-imagery_eeg.edf]: 0it [00:00, ?it/s]
[RECORD ./strokedataset_TEST/edffile/sub-12/eeg/sub-12_task-motor-imagery_eeg.edf]: 1it [00:00,  8.98it/s]
                                                                                                          [2024-11-22 15:29:26] INFO (torcheeg/MainThread) ✅ | All processed EEG data has been cached to .torcheeg/datasets_1732260564718_uh536.
[2024-11-22 15:29:26] INFO (torcheeg/MainThread) 😊 | Please set io_path to .torcheeg/datasets_1732260564718_uh536 for the next run, to d

320


In [14]:
import torch
from model import SwinTransformer
model_weight_path = "./weights/model-77-9_0876.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = SwinTransformer(
#                         img_size=9,           # 输入的空间尺寸
#                         patch_size=3,         # 每个补丁为 3x3
#                         in_chans=5,           # 通道数为4
#                         num_classes=3,        # 3分类
#                         window_size=3         # 窗口大小3x3
#                     )
model = SwinTransformer(
                        img_size=7,           # 输入的空间尺寸
                        patch_size=7,         # 每个补丁为 3x3
                        in_chans=500,           # 通道数为4
                        num_classes=2,        # 3分类
                        window_size=3         # 窗口大小3x3
                    ).to(device)
model.load_state_dict(torch.load(model_weight_path, map_location=device))
model.eval()

correct = 0  # 记录正确预测的数量
total = 0    # 记录总的预测数量

for i, data_pre in enumerate(predict_datalist):
    with torch.no_grad():
        # 预测类别
        data_pre = data_pre.to(device)
        output = torch.squeeze(model(data_pre)).to(device)  # 注意：data_pre应是当前数据，而不是data_predict
        predict = torch.softmax(output, dim=0).to(device)
        predict_cla = torch.argmax(predict).cpu().numpy()
    
    # 判断预测是否正确
    if predict_cla == predict_labellist[i]:
        correct += 1  # 如果预测正确，增加计数

    total += 1  # 每次预测后，总数加1

# 计算准确率
accuracy = correct / total * 100  # 以百分比形式表示准确率
print(f'Accuracy: {accuracy:.2f}%')


Accuracy: 0.00%
